In [ ]:
import sys

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   import ee
   #ee.Authenticate()
   ee.Initialize()
else:
   print('GEE already imported')
   # google earth engine already imported and authenticated


modulename = 'ipynb_masks'
if modulename not in sys.modules:
    %run Masks.ipynb
    sys.modules['ipynb_masks'] = None
#else
    # module already loaded

modulename = 'ipynb_Utils'
if modulename not in sys.modules:
    %run Utils.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Utils'] = None 
#else
   # Utils modules has already been loaded somewhere else



In [ ]:
from datetime import date

#Define global variables related to Sentinel-1
sentinel1Info = {
  "collection": "COPERNICUS/S1_GRD",
  "startYear" : "2014-04-01",
  "endDate"   : None
}

## method that returns the name of the collection used in the Sentinel1 class
def getSentinel1Col():
    return sentinel1Info["collection"]
 
## method that sets the Sentinel1 collection 
#  this is useful in case an updated class is released in GEE or for consistency an 
#  older version of Sentinel1 col needs to be used
# @param[in] newcol: the name of the collection that will replace the collection 
# used in this class
# @note: Please use with caution as a wrong collection will crash the entire class
def setSentinel1Col(newCol):
    global sentinel1Info
    sentinel1Info["collection"] = newCol

## method that returns the the start end end date of the collection
#  @param[in] type 'whole' or 'year' get the entire date or just the year
def getStartEndDateS1(type):
  startDate = sentinel1Info["startYear"]
  endDate   = sentinel1Info["endDate"  ]
  if(endDate == None):
    today = date.today()
    endDate = today.strftime("%Y-%m-%d")
  if type == 'year' :
     return [getYearMonthOrDay(startDate,'year'),getYearMonthOrDay(endDate,'year')]
  elif type == 'whole':
     return [startDate,endDate]
  else: # type not defined
     raise Exception("ERROR: type for getting start end end date/year of Sentinel-1 not defined\n")
     


In [ ]:
class Sentinel1:
    """
    # @brief method that takes as input a collection, calculates pixelwise average for each month and returns a new collection
    def byMonthIndermidateFunction(self,m):
        phenCol =  self.sentinel1.filter(ee.Filter.calendarRange(m, m, 'month')).mean().set('month', m)        
        return phenCol
    
    
    # @param[in] collection : the collection to be processed
    # @return a collection annual monthly average pixel values

    def byMonth (self):
        months = ee.List.sequence(1, 12)
        return ee.ImageCollection.fromImages(months.map(algorithm = self.byMonthIndermidateFunction))
    """
    def renameVVAsc(self,img):
        return img.rename('VVAsc')
        
    def renameVHAsc(self,img):
        return img.rename('VHAsc')

    def renameVVDes(self,img):
        return img.rename('VVDes')

    def renameVHDes(self,img):
        return img.rename('VHDes')
    
    # @brief initialisation of Sentinel1 class
    # @param[in] geometry: area of interest
    # @param[in] startDate: start date of the collection to be retrived
    # @param[in] endDate: end date of the collection to be retrived
    # @param[in] cloudfilter: percentage of non acceptable clouds for removing images
    # @param[in] masks: a dictonary containing the masks to be applied 
    # @param[in] applyAspects: is a boolean that defines whether the corresponding ascpets will be applied
    # Format of masks should be the following, but it should include only the masks 
    # that we want to be applied
    # masks = {'gsw':buffer, 'lmask': buffer, 'forestMask': {buffer,year}}
    def __init__(self,geometry, startDate,endDate, masks, applyAspects):
        self.startDate = startDate
        self.endDate   = endDate
        self.sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
                           .filterDate(startDate,endDate) \
                           .filterBounds(geometry) \
                           .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV')) \
                           .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH')) \
                           .filter(ee.Filter.eq('instrumentMode', 'IW'))

        # apply water and land mask
        # create forest lost mask 
        #buffer = 30

        year = ee.Date(endDate).get('year') 
        masksHandler = Masks(geometry,masks)
        masksHandler.calculateCombinedMask() # combined mask does not include aspects
        self.sentinel1 = self.sentinel1.map(algorithm = masksHandler.updateCombinedMask)
        
        self.VVAsc     = self.sentinel1.filter(ee.Filter.eq('orbitProperties_pass','ASCENDING' )) \
                                                        .select('VV') \
                                                        .map(filterSpeckles3x3) \
                                                        .map(algorithm = self.renameVVAsc)
        self.VHAsc     = self.sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING' )) \
                                                        .select('VH') \
                                                        .map(filterSpeckles3x3) \
                                                        .map(algorithm = self.renameVHAsc)
        self.VVDes     = self.sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
                                                        .select('VV') \
                                                        .map(filterSpeckles3x3) \
                                                        .map(algorithm = self.renameVVDes)
        self.VHDes     = self.sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).select('VH') \
                                                        .map(filterSpeckles3x3) \
                                                        .map(algorithm = self.renameVHDes)
        self.tmpCol    = None
        self.tmpyear   = 0

        if(applyAspects):
            masksHandlerAscAsp = Masks(geometry,{'aspectAsc':0})
            masksHandlerDesAsp = Masks(geometry,{'aspectDes':0})
            masksHandlerAscAsp.calculateCombinedMask() 
            masksHandlerDesAsp.calculateCombinedMask() 
            self.VVAsc = self.VVAsc.map(algorithm = masksHandlerAscAsp.updateAscMask)
            self.VHAsc = self.VHAsc.map(algorithm = masksHandlerAscAsp.updateAscMask) 
            self.VVDes = self.VVDes.map(algorithm = masksHandlerDesAsp.updateDesMask) 
            self.VHDes = self.VHDes.map(algorithm = masksHandlerDesAsp.updateDesMask)

        years  = [2017]
        months = list(range(1,13))
        #self.VHDes = monthly_Avg(self.VHDes,years,months)
        #self.VHDes     = byMonth(self.VHDes)
        #self.byMonthVHDesVHDes()

    def getVVAsc(self):
        return self.VVAsc

    def getVHAsc(self):
        return self.VHAsc

    def getVVDes(self):
        return self.VVDes

    def getCollectionToBands(self):
        VHAscBands = self.VHAsc.toBands()
        VVAscBands = self.VVAsc.toBands()
        VHDesBands = self.VHDes.toBands()
        VVDesBands = self.VVDes.toBands()
        return VHAscBands.addBands(VVAscBands).addBands(VHDesBands).addBands(VVDesBands)
    
    def getCollectionToBandsAsc(self):
        VHAscBands = self.VHAsc.toBands()
        VVAscBands = self.VVAsc.toBands()
        return VHAscBands.addBands(VVAscBands)
    
    def getCollectionToBandsDes(self):
        VHDesBands = self.VHDes.toBands()
        VVDesBands = self.VVDes.toBands()
        return VHDesBands.addBands(VVDesBands)



    def getVHDes(self):
        return self.VHDes 
    
    def getVVAscMedian(self):
        return self.VVAsc.median()

    def getVHAscMedian(self):
        return self.VHAsc.median()

    def getVVDesMedian(self):
        return self.VVDes.median()

    def getVHDesMedian(self):
        return self.VHDes.median()
    
    
    def getAveOfMonthVHDes(self,m): 
        return  self.VHDes.filter(ee.Filter.calendarRange(m, m, 'month')) #\
                  #  .mean() \
                   # .set('month', m) 

    def byMonthVHDesVHDes(self,i_year,col):
        startDate = ee.Date.fromYMD(i_year, 1, 1)
        endDate = startDate.advance(1, 'year')
        tmpCol = col.filter(ee.Filter.date(startDate, endDate))
        return  ee.ImageCollection.fromImages(ee.List([
        (tmpCol.filter(ee.Filter.calendarRange( 1,  1, 'month')).mean().set('month', 1)),
        (tmpCol.filter(ee.Filter.calendarRange( 2,  2, 'month')).mean().set('month', 2)),
        (tmpCol.filter(ee.Filter.calendarRange( 3,  3, 'month')).mean().set('month', 3)),
        (tmpCol.filter(ee.Filter.calendarRange( 4,  4, 'month')).mean().set('month', 4)),
        (tmpCol.filter(ee.Filter.calendarRange( 5,  5, 'month')).mean().set('month', 5)),
        (tmpCol.filter(ee.Filter.calendarRange( 6,  6, 'month')).mean().set('month', 6)),
        (tmpCol.filter(ee.Filter.calendarRange( 7,  7, 'month')).mean().set('month', 7)),
        (tmpCol.filter(ee.Filter.calendarRange( 8,  8, 'month')).mean().set('month', 8)),
        (tmpCol.filter(ee.Filter.calendarRange( 9,  9, 'month')).mean().set('month', 9)),
        (tmpCol.filter(ee.Filter.calendarRange(10, 10, 'month')).mean().set('month',10)),
        (tmpCol.filter(ee.Filter.calendarRange(11, 11, 'month')).mean().set('month',11)),
        (tmpCol.filter(ee.Filter.calendarRange(12, 12, 'month')).mean().set('month',12))]))
    
    def byMonth(self,i_year):
        self.VHAsc = self.byMonthVHDesVHDes(i_year,self.VHAsc)
        self.VVAsc = self.byMonthVHDesVHDes(i_year,self.VVAsc)
        self.VHDes = self.byMonthVHDesVHDes(i_year,self.VHDes)
        self.VVDes = self.byMonthVHDesVHDes(i_year,self.VVDes)
                    
    # method that removed a given period from the dataset
    def removePeriod(self,startDate, endDate):
        badDataFilter = ee.Filter.date(startDate,endDate)
        self.VHAsc = self.VHAsc.filter(badDataFilter.Not())
        self.VVAsc = self.VVAsc.filter(badDataFilter.Not())
        self.VHDes = self.VHDes.filter(badDataFilter.Not())
        self.VVDes = self.VVDes.filter(badDataFilter.Not())
        print("Period from ", startDate, " to ", endDate, " removed")

In [ ]:
print ("Sentinel1 class imported")